In [3]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from ipywidgets import interact_manual


distance = ctrl.Antecedent(np.arange(0, 140, 1), 'distance')
color = ctrl.Antecedent(np.arange(0, 6, 1), 'color')
speed = ctrl.Antecedent(np.arange(0, 200, 1), 'speed')
delta = ctrl.Consequent(np.arange(0, 200, 1), 'delta')

distance.automf(names=["far","mid", "near"])

speed['zero'] = fuzz.trimf(speed.universe, [0, 0, 0])
speed['slow'] = fuzz.trimf(speed.universe, [0, 20, 40])
speed['can_pass'] = fuzz.trimf(speed.universe, [20, 40, 60])
speed['comfortable'] = fuzz.trimf(speed.universe, [40, 60, 80])
speed['too_fast'] = fuzz.trimf(speed.universe, [60, 80, 100])

color['red'] = fuzz.trimf(color.universe, [0, 0, 1])
color['yellow'] = fuzz.trimf(color.universe, [0, 1, 2])
color['yellow_flash'] = fuzz.trimf(color.universe, [1, 2, 3])
color['green_flash'] = fuzz.trimf(color.universe, [2, 3,4])
color['green'] = fuzz.trimf(color.universe, [3, 4, 5])
color['broken'] = fuzz.trimf(color.universe, [4, 5, 6])

delta['zero'] = fuzz.trimf(delta.universe, [0, 0, 0])
delta['slow'] = fuzz.trimf(delta.universe, [0, 20, 40])
delta['can_pass'] = fuzz.trimf(delta.universe, [20, 40, 60])
delta['comfortable'] = fuzz.trimf(delta.universe, [40, 60, 80])

rule1 = ctrl.Rule(distance['far'] & (speed['too_fast']), delta['comfortable'])
rule11 = ctrl.Rule((distance['mid'] | distance['near']) & speed['too_fast'] , delta['can_pass'])
rule12 = ctrl.Rule(distance['mid'] & color['green'] & speed['slow'] , delta['comfortable'])
rule13 = ctrl.Rule(distance['mid'] & color['red'] , delta['can_pass'])
rule14 = ctrl.Rule(distance['mid'] & color['green_flash'] , delta['slow'])
rule2 = ctrl.Rule(distance['near'] & color['red'] , delta['zero'])
rule3 = ctrl.Rule(distance['near'] & (color['yellow'] | color['yellow_flash'] | color['green_flash']), delta['can_pass'])
rule4 = ctrl.Rule(distance['near'] & color['green'], delta['can_pass'])

@interact_manual
def get_input(speed=100, color=3, distance = 100):
    tipping_ctrl = ctrl.ControlSystem([rule1, rule11,rule13, rule14,rule12, rule2, rule3, rule4])
    tipping = ctrl.ControlSystemSimulation(tipping_ctrl)
    tipping.input['color'] = color
    tipping.input['distance'] = distance
    tipping.input['speed'] = speed
    tipping.compute()
    print('Recommended speed',tipping.output['delta'])


A Jupyter Widget